In [3]:
import tensorflow as tf
import numpy as np
import os
print(tf.__version__)

2.0.0


In [21]:
num_epochs = 30
batch_size = 128
learning_rate = 0.001
data_dir = './data/fer2013/'
# 0 anger 生气； 1 disgust 厌恶； 2 fear 恐惧； 3 happy 开心； 4 sad 伤心；5 surprised 惊讶； 6 normal 中性
train_anger_dir = data_dir + 'train/0/'
train_disgust_dir = data_dir + 'train/1/'
train_fear_dir = data_dir + 'train/2/'
train_happy_dir = data_dir + 'train/3/'
train_sad_dir = data_dir + 'train/4/'
train_surprised_dir = data_dir + 'train/5/'
train_normal_dir = data_dir + 'train/6/'

test_anger_dir = data_dir + 'valid/0/'
test_disgust_dir = data_dir + 'valid/1/'
test_fear_dir = data_dir + 'valid/2/'
test_happy_dir = data_dir + 'valid/3/'
test_sad_dir = data_dir + 'valid/4/'
test_surprised_dir = data_dir + 'valid/5/'
test_normal_dir = data_dir + 'valid/6/'

checkpoint_path = "training_fer2013_CNN_1/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

def _decode_and_resize(filename, label):
    image_string = tf.io.read_file(filename)                   # 读取原始文件
    image_decoded = tf.image.decode_jpeg(image_string)         # 解码JPEG图片
    image_resized = tf.image.resize(image_decoded, [48, 48]) / 255.0
    return image_resized, label

class CNN(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv11 = tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=[1, 1],
        strides=(1, 1),
        padding='same',
        activation=tf.nn.relu
    )
    self.conv12 = tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=[3, 3],
        strides=(1, 1),
        padding='same',
        activation=tf.nn.relu
    )
    self.conv13 = tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=[5, 5],
        strides=(1, 1),
        padding='same',
        activation=tf.nn.relu
    )
    self.concatenate = tf.keras.layers.Concatenate()
    self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
    self.conv2 = tf.keras.layers.Conv2D(
        filters=64,
        kernel_size=[3, 3],
        strides=(1, 1),
        padding='same',
        activation=tf.nn.relu
    )
    self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
    self.conv3 = tf.keras.layers.Conv2D(
        filters=128,
        kernel_size=[3, 3],
        strides=(1, 1),
        padding='same',
        activation=tf.nn.relu
    )
    self.pool3 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
    self.flatten = tf.keras.layers.Reshape(target_shape=(6 * 6 * 128,))
    self.dense1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(units=7)

  def call(self, inputs):
    x1 = self.conv11(inputs)  # [48, 48, 32]
    x2 = self.conv12(inputs)  # [48, 48, 32]
    x3 = self.conv13(inputs)  # [48, 48, 32]
    x = self.concatenate([x1, x2, x3]) # [48, 48, 96]
    x = self.pool1(x)    # [24, 24, 96]
    x = self.conv2(x)    # [24, 24, 64]
    x = self.pool2(x)    # [12, 12, 64]
    x = self.conv3(x)    # [12, 12, 128]
    x = self.pool3(x)    # [6, 6, 128]
    x = self.flatten(x)   # [6*6*128]
    x = self.dense1(x)    # [256]
    x = self.dense2(x)    # [7]
    output = tf.nn.softmax(x)
    return output

def train():
    # 构建训练数据集
    # 0 anger 生气； 1 disgust 厌恶； 2 fear 恐惧； 3 happy 开心； 4 sad 伤心；5 surprised 惊讶； 6 normal 中性
    train_anger_filenames = tf.constant([train_anger_dir + filename for filename in os.listdir(train_anger_dir)])
    train_disgust_filenames = tf.constant([train_disgust_dir + filename for filename in os.listdir(train_disgust_dir)])
    train_fear_filenames = tf.constant([train_fear_dir + filename for filename in os.listdir(train_fear_dir)])
    train_happy_filenames = tf.constant([train_happy_dir + filename for filename in os.listdir(train_happy_dir)])
    train_sad_filenames = tf.constant([train_sad_dir + filename for filename in os.listdir(train_sad_dir)])
    train_surprised_filenames = tf.constant([train_surprised_dir + filename for filename in os.listdir(train_surprised_dir)])
    train_nromal_filenames = tf.constant([train_normal_dir + filename for filename in os.listdir(train_normal_dir)])
    train_filenames = tf.concat([train_anger_filenames, train_disgust_filenames, train_fear_filenames, train_happy_filenames, train_sad_filenames, train_surprised_filenames, train_nromal_filenames], axis=-1)
    train_labels = tf.concat([
        tf.zeros(train_anger_filenames.shape, dtype=tf.int32),
        tf.ones(train_disgust_filenames.shape, dtype=tf.int32),
        tf.ones(train_fear_filenames.shape, dtype=tf.int32) * 2,
        tf.ones(train_happy_filenames.shape, dtype=tf.int32) * 3,
        tf.ones(train_sad_filenames.shape, dtype=tf.int32) * 4,
        tf.ones(train_surprised_filenames.shape, dtype=tf.int32) * 5,
        tf.ones(train_nromal_filenames.shape, dtype=tf.int32) * 6],
        axis=-1
    )
    
    train_dataset = tf.data.Dataset.from_tensor_slices((train_filenames, train_labels))
    train_dataset = train_dataset.map(
        map_func = _decode_and_resize,
        num_parallel_calls = tf.data.experimental.AUTOTUNE
    )# 取出前buffer_size个数据放入buffer，并从其中随机采样，采样后的数据用后续数据替换
    train_dataset = train_dataset.shuffle(buffer_size=23000)
    train_dataset = train_dataset.batch(batch_size)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    model.fit(train_dataset, epochs=num_epochs, callbacks=[cp_callback_mc])

def test():
    # 构建测试数据集
    test_anger_filenames = tf.constant([test_anger_dir + filename for filename in os.listdir(test_anger_dir)])
    test_disgust_filenames = tf.constant([test_disgust_dir + filename for filename in os.listdir(test_disgust_dir)])
    test_fear_filenames = tf.constant([test_fear_dir + filename for filename in os.listdir(test_fear_dir)])
    test_happy_filenames = tf.constant([test_happy_dir + filename for filename in os.listdir(test_happy_dir)])
    test_sad_filenames = tf.constant([test_sad_dir + filename for filename in os.listdir(test_sad_dir)])
    test_surprised_filenames = tf.constant([test_surprised_dir + filename for filename in os.listdir(test_surprised_dir)])
    test_nromal_filenames = tf.constant([test_normal_dir + filename for filename in os.listdir(test_normal_dir)])
    test_filenames = tf.concat([test_anger_filenames, test_disgust_filenames, test_fear_filenames, test_happy_filenames, test_sad_filenames, test_surprised_filenames, test_nromal_filenames], axis=-1)
    test_labels = tf.concat([
        tf.zeros(test_anger_filenames.shape, dtype=tf.int32),
        tf.ones(test_disgust_filenames.shape, dtype=tf.int32),
        tf.ones(test_fear_filenames.shape, dtype=tf.int32) * 2,
        tf.ones(test_happy_filenames.shape, dtype=tf.int32) * 3,
        tf.ones(test_sad_filenames.shape, dtype=tf.int32) * 4,
        tf.ones(test_surprised_filenames.shape, dtype=tf.int32) * 5,
        tf.ones(test_nromal_filenames.shape, dtype=tf.int32) * 6],
        axis=-1
    )

    test_dataset = tf.data.Dataset.from_tensor_slices((test_filenames, test_labels))
    test_dataset = test_dataset.map(_decode_and_resize)
    test_dataset = test_dataset.batch(batch_size)
    
    print(model.evaluate(test_dataset))
    
if __name__ ==  '__main__':
    model = CNN()
    model.build(input_shape=(None, 48, 48, 1))
    model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                    loss=tf.keras.losses.sparse_categorical_crossentropy,
                    metrics=[tf.keras.metrics.sparse_categorical_accuracy])
    
    cp_callback_mc = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                        save_weights_only=True,
                                                        verbose=0)
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    if(latest != None):
        model.load_weights(latest) 

Model: "cnn_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_34 (Conv2D)           multiple                  64        
_________________________________________________________________
conv2d_35 (Conv2D)           multiple                  320       
_________________________________________________________________
conv2d_36 (Conv2D)           multiple                  832       
_________________________________________________________________
concatenate_8 (Concatenate)  multiple                  0         
_________________________________________________________________
max_pooling2d_18 (MaxPooling multiple                  0         
_________________________________________________________________
conv2d_37 (Conv2D)           multiple                  55360     
_________________________________________________________________
max_pooling2d_19 (MaxPooling multiple                  0     

In [22]:
    train()

Epoch 1/30
225/225 [==============================] - 159s 708ms/step - loss: 0.1011 - sparse_categorical_accuracy: 0.9707
Epoch 2/30
225/225 [==============================] - 161s 713ms/step - loss: 0.0844 - sparse_categorical_accuracy: 0.9765
Epoch 3/30
225/225 [==============================] - 164s 730ms/step - loss: 0.0772 - sparse_categorical_accuracy: 0.9792
Epoch 4/30
225/225 [==============================] - 169s 752ms/step - loss: 0.0673 - sparse_categorical_accuracy: 0.9820
Epoch 5/30
225/225 [==============================] - 166s 738ms/step - loss: 0.0637 - sparse_categorical_accuracy: 0.9827
Epoch 6/30
225/225 [==============================] - 159s 709ms/step - loss: 0.0828 - sparse_categorical_accuracy: 0.9759
Epoch 7/30
225/225 [==============================] - 160s 711ms/step - loss: 0.0610 - sparse_categorical_accuracy: 0.9834
Epoch 8/30
225/225 [==============================] - 160s 713ms/step - loss: 0.0606 - sparse_categorical_accuracy: 0.9842
Epoch 9/30
225/2

In [23]:
    test()

29/29 [==============================] - 7s 242ms/step - loss: 4.2449 - sparse_categorical_accuracy: 0.5464
[4.2448750611009265, 0.5463917]
